In [1]:
from Libreria.cargar_configbd import cargar_configbd
import geopandas as gpd
import pandas as pd
from Libreria.mapa_D_colores import mapa_D_colores
import pyproj

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [ ]:
# ---------------------------------------RUTAS BUSES

In [5]:
# Crear tabla para insertar rutas de buses
consultas_sql = ["DROP TABLE IF EXISTS rutas_bus_cuenca;",
    """CREATE TABLE IF NOT EXISTS rutas_bus_cuenca (
        latitud_Y NUMERIC,
        longitud_X NUMERIC,
        SECUENCIA INTEGER,
        Origen INTEGER,
        Destino INTEGER,
        ID INTEGER,
        wkt_geom TEXT,
        ID_LINEA INTEGER,
        LINEA TEXT,
        COMPAÑIA TEXT,
        SENTIDO TEXT,
        "Nro Ruta" INTEGER,
        Alargue INTEGER,
        "ID-PARADA" INTEGER,
        PARADA TEXT,
        "ID-PROX PA" INTEGER,
        PARADA2 TEXT,
        latitud2 NUMERIC,
        longitud2 NUMERIC,
        Distancia INTEGER,
        "Tiempo seg" NUMERIC,
        "Tiempo min" NUMERIC,
        Correcciin NUMERIC,
        FINAL NUMERIC,
        Observacio TEXT,
        HubName INTEGER,
        HubDist NUMERIC,
        Coronas TEXT,
        V_Bus NUMERIC,
        V_Tranvia NUMERIC,
        geometry_ls TEXT
    );
    """]

for consulta_sql in consultas_sql:
    cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [6]:
# Path al archivo GIS Rutas Buses
rb = "Datos/Rutas Bus/OD_RutasCuenca_RedCompleta.shp"
#rb = "Datos/Rutas Bus/RutasCuenca_RedCompleta.shp"

# Read GIS data
# Try reading the file with different encodings and drivers
try:
    df_rb = gpd.read_file(rb)
except UnicodeDecodeError:
    try:
        # Try reading with latin1 encoding
        df_rb = gpd.read_file(rb, encoding='latin1')
    except UnicodeDecodeError:
        # Try reading with ISO-8859-1 encoding
        df_rb = gpd.read_file(rb, encoding='ISO-8859-1')
    except:
        print("Error reading the file with alternative encodings.")
        
df_rb.rename(columns={'latitud_Y': 'latitude', 'longitud_X': 'longitude'}, inplace=True)

In [7]:

# Tabla para insertar registros GIS
nombre_tabla = "rutas_bus_cuenca"

print(f"n_puntos: {len(df_rb)}")
prog = 0   
# Recorre el DataFrame y inserta cada fila en la tabla
for index, row in df_rb.iterrows():
    prog += 1
    print(f"Progreso: {prog}", end="\r")

    # Convierte la geometría a formato WKT (Well-Known Text)
    wkt_geom = row['geometry'].wkt
    
    # Construye la consulta SQL con parámetros
    consulta_sql = """
    INSERT INTO {} VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s
    )
    """.format(nombre_tabla)
    
    # Ejecuta la consulta SQL con los valores de la fila actual
    cur.execute(consulta_sql, tuple(row)[:-1] + (wkt_geom,))

'''cur.execute(f"ALTER TABLE {nombre_tabla} ADD COLUMN geometry geometry(Point, 4326);")
cur.execute(f"UPDATE {nombre_tabla} SET geometry = ST_SetSRID(ST_MakePoint(longitud_x, latitud_y), 4326);")'''

# Confirma los cambios en la base de datos
conn.commit()

n_puntos: 4543


In [1]:
#mapa_D_colores.mapa(df_rb, 'latitude', 'longitude', 'ID-LINEA')

In [ ]:
# ---------------------------- RUTAS CICLOVIAS

In [11]:
# Crear tabla para insertar rutas de ciclovias
consultas_sql = ["DROP TABLE IF EXISTS ciclovias_cuenca;",
    """CREATE TABLE IF NOT EXISTS ciclovias_cuenca (
        id INTEGER,
        Nombre TEXT,
        Distancia FLOAT,
        Tipo TEXT,
        Sentido TEXT,
        Ancho_prom TEXT,
        latitud NUMERIC,
        longitud NUMERIC,
        secuencia INTEGER
    );
    """]

for consulta_sql in consultas_sql:
    cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [12]:
# Path al archivo GIS
ic2024 = "Datos/Ciclovias/Infraestructura Ciclista 2024.shp"

# Read GIS data
df_ic2024 = gpd.read_file(ic2024, encoding='ISO-8859-1')

In [13]:
# Definir los sistemas de coordenadas de origen (EPSG:32717) y destino (EPSG:4326)
src_crs = pyproj.CRS.from_epsg(32717)
dst_crs = pyproj.CRS.from_epsg(4326)

# Crear un transformador de coordenadas
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

# Tabla para insertar registros GIS
nombre_tabla = "ciclovias_cuenca"
print(f"n_ciclovias: {len(df_ic2024)}")
prog = 0
for index, ciclovia in df_ic2024.iterrows():
    prog += 1
    print(f"Progreso: {prog}", end="\r")
    

    # Convierte la geometría a formato WKT (Well-Known Text)
    secuencia = 0
    for point in ciclovia['geometry'].coords:
        x, y = point
        latitud, longitud = transformer.transform(x, y)

        # Construye la consulta SQL con parámetros
        consulta_sql = """
        INSERT INTO {} VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """.format(nombre_tabla)
        secuencia += 1
        # Ejecuta la consulta SQL con los valores de la fila actual
        cur.execute(consulta_sql, (index+1, ) + tuple(ciclovia[['Nombre', 'Distancia', 'Tipo', 'Sentido', 'Ancho prom']]) + (latitud, longitud, secuencia))

# Confirma los cambios en la base de datos
conn.commit()

n_ciclovias: 45


In [14]:

# Obtener los ids de todos los viajes
cur.execute(f"SELECT * FROM ciclovias_cuenca")
# Obtener los resultados de la consulta
rows = cur.fetchall()
puntos_ciclovias = rows.copy()

# Crear un DataFrame a partir de la lista de tuplas
df_ic2024 = pd.DataFrame(puntos_ciclovias, columns=[desc[0] for desc in cur.description])

In [15]:
mapa_D_colores.mapa(df_ic2024, 'latitud', 'longitud', 'id')